# Main Streamlit HomePage
5 pages:
- Top rated movies with filtering 
- Item-Item recommendations
- Personalized recommendations
- User profile creation: provide ratings
- Visualizations   

In [1]:
import streamlit as st 
import pandas as pd
import os
import numpy as np
import operator
import fastparquet
import re
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

### Import Individual Pages

In [ ]:
import pages.home_page
import pages.non_user_recommendations
import pages.item_item_rec_app
import pages.personalized_rec_app
import pages.profile_add_app

## Set up data and unique lists for filtering 
Needed for multiple of the pages, so more efficient to only load once

In [3]:
@st.cache(allow_output_mutation=True)
def data_setup():
    # read in data created in recommendation_data_display.ipynb
    df = pd.read_parquet('recommendation_display.parq')
    
    # recombine genre lists to string for tf-idf
    df['genre_str'] = df.Genres.apply(lambda row: ' '.join(row))

    # get unique lists of all filter values
    genres_unique, actors_df, directors_df, countries_unique, language_unique, tags_unique = pages.non_user_recommendations.unique_lists(df)
    
    return df, genres_unique, actors_df, directors_df, countries_unique, language_unique, tags_unique

In [1]:
df, genres_unique, actors_df, directors_df, countries_unique, language_unique, tags_unique = data_setup()

## Run cached set up functions

In [ ]:
movieIds, indices, tfidf_matrix, movies_unique = pages.item_item_rec_app.cached_functions(df)

In [ ]:
df_dummies, ratings_lst, user_lst, movies_lst = pages.personalized_rec_app.load_data()

# Main Function: Navigation between Pages
- Side radio button. Upon selection, call write() function within each page. Pass in arguments from cached calls. 

In [ ]:
PAGES = ['Home', 'Top Rated Movies', 'Movie Based Recommendations', 'Personalized Recommendations', 'Add Profile']

In [12]:
def main(df, genres_unique, actors_df, directors_df, countries_unique, language_unique, tags_unique,
         movieIds, indices, tfidf_matrix, movies_unique, df_dummies, ratings_lst, user_lst, movies_lst):
    
    st.sidebar.title("Navigation")
    selection = st.sidebar.radio("Go to", PAGES)    
    
    if selection == 'Home':
        pages.home_page.write()
    if selection == 'Top Rated Movies':
        pages.non_user_recommendations.write(df, genres_unique, actors_df, 
                                             directors_df, countries_unique, language_unique, tags_unique)
    if selection == 'Movie Based Recommendations':
        pages.item_item_rec_app.write(df, movieIds, indices, tfidf_matrix, movies_unique)
    if selection == 'Personalized Recommendations':
        pages.personalized_rec_app.write(df, genres_unique, actors_df, directors_df, countries_unique,
                                          language_unique, tags_unique, ratings_lst, user_lst, movies_lst, df_dummies)
    if selection == 'Add Profile':
        #st.write(ratings.userId.max())
        ratings_lst, user_lst, movies_lst = pages.profile_add_app.write(df, ratings_lst, user_lst, movies_lst)
        #st.write(ratings.userId.max())
        
    return ratings_lst, user_lst, movies_lst

In [ ]:
ratings_lst, user_lst, movies_lst = main(df, genres_unique, actors_df, directors_df, countries_unique,
                                         language_unique, tags_unique,
                                         movieIds, indices, tfidf_matrix, movies_unique, df_dummies, 
                                         ratings_lst, user_lst, movies_lst)